In [ ]:
!pip install transformers
from huggingface_hub import notebook_login
notebook_login()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 21.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
import re
import math
from transformers import BertTokenizer, BertModel
from google.colab import drive
drive.mount('/content/gdrive')

# Run locally
#DATA_PATH = ''

# KW
DATA_PATH = '/content/gdrive/MyDrive/CSI6900/'

# FZ
#DATA_PATH = 'gdrive/MyDrive/CSI6900/'

device = torch.device("mps" if getattr(torch,'has_mps',False) else "cuda:0" if torch.cuda.is_available() else "cpu")
device

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


device(type='cuda', index=0)

## Load data

In [ ]:
records = pd.read_json(DATA_PATH + 'data/records.json')
records = records.drop(['created_at', 'updated_at', 'deleted_at', 'publish', 'academic_credentials', 'age_max', 'age_min', 'last_name', 
                        'latitude', 'longitude', 'name_of_private_practice', 'fee_description',	'fee_type',	'first_name',	'languages',
                        'organization_type', 'original_id',	'record_type',	'salutation_type', 'website'], axis=1)

taxonomy = pd.read_json(DATA_PATH + 'data/taxonomy_headings.json')
taxonomy = taxonomy.drop(['created_at',	'updated_at',	'deleted_at', 'alias_of_id', 'short_description',	'original_id'], axis=1)

taxonomy.head(10)

,id,name,description,translations
0,1,Root,Root,"{""name"":{""en"":""Root"",""fr"":null},""description"":..."
1,2,All Mental Health Resources,<p>\r\n\tThe listings of mental health resourc...,"{""name"":{""en"":""All Mental Health Resources"",""f..."
2,3,Crisis and Emergency,<p>\r\n\tRefers to all programs that provide i...,"{""name"":{""en"":""Crisis and Emergency"",""fr"":""Res..."
3,4,"System Navigation, including Information and R...","<p>\r\n\tAre you looking for help, but don&#39...","{""name"":{""en"":""System Navigation, including In..."
4,5,Child Welfare including Children's Aid Society...,<p>The child welfare / child protection system...,"{""name"":{""en"":""Child Welfare including Childre..."
5,6,Emergency Shelter and Housing,<p>\r\n\tThere are various shelters that peopl...,"{""name"":{""en"":""Emergency Shelter and Housing"",..."
6,7,Hospital Emergency Department,<p>\r\n\tIs there an emergency such as medical...,"{""name"":{""en"":""Hospital Emergency Department"",..."
7,8,"Crisis Lines including Telephone, Online and Chat",<p>\r\n\tAre you in a crisis? Crisis lines off...,"{""name"":{""en"":""Crisis Lines including Telephon..."
8,9,Psychiatrists,<p>\r\n\tPsychiatrists are medical doctors who...,"{""name"":{""en"":""Psychiatrists"",""fr"":""Psychiatre..."
9,10,A-Z Mental Health Conditions and Topics,<p>\r\n\tAlphabetical list of mental health to...,"{""name"":{""en"":""A-Z Mental Health Conditions an..."


## Pre-processing

In [ ]:
!cp /content/gdrive/MyDrive/CSI6900/pre_processing.py /content

In [ ]:
from pre_processing import PreProcessing

records, taxonomy = PreProcessing(records, taxonomy).preprocess()

Length of records before preprocessing: 6406
Length of taxonomy before preprocessing: 277
Length of records after preprocessing: 6239
Length of taxonomy after preprocessing: 192


## Generate embeddings

In [ ]:
tokenizer = BertTokenizer.from_pretrained("AIMH/mental-bert-large-uncased")
model = BertModel.from_pretrained("AIMH/mental-bert-large-uncased").to(device)
model.eval()

Some weights of the model checkpoint at AIMH/mental-bert-large-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inpl

In [ ]:
records_description = records['description'].tolist()
tokenized_record = tokenizer(records_description, padding='max_length', max_length=512, truncation=True, return_tensors='pt').to(device)
taxonomy_description = taxonomy['description'].tolist()
tokenized_taxonomy = tokenizer(taxonomy_description, padding='max_length', max_length=512, truncation=True, return_tensors='pt').to(device)

In [ ]:
batch_size = 128

num_batches = math.ceil(len(tokenized_record.input_ids)/batch_size)

# Generate embeddings for the tokenized_record using the BERT model in batches
records_embeddings = []
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size
    end_idx = end_idx if end_idx <= len(tokenized_record.input_ids) else len(tokenized_record.input_ids)

    batch_input_ids = tokenized_record['input_ids'][start_idx:end_idx].to(device)
    batch_attention_mask = tokenized_record['attention_mask'][start_idx:end_idx].to(device)
    batch_token_type_ids = tokenized_record['token_type_ids'][start_idx:end_idx].to(device)

    with torch.no_grad():
        embedding = model(input_ids=batch_input_ids,
                        attention_mask=batch_attention_mask,
                        token_type_ids=batch_token_type_ids)
        embedding = embedding.last_hidden_state.mean(dim=1).cpu()
    
    records_embeddings.append(embedding)

In [ ]:
batch_size = 128

num_batches = math.ceil(len(tokenized_taxonomy.input_ids)/batch_size)

# Generate embeddings for the tokenized_taxonomy using the BERT model in batches
taxonomy_embeddings = []
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size
    end_idx = end_idx if end_idx <= len(tokenized_taxonomy.input_ids) else len(tokenized_taxonomy.input_ids)

    batch_input_ids = tokenized_taxonomy['input_ids'][start_idx:end_idx].to(device)
    batch_attention_mask = tokenized_taxonomy['attention_mask'][start_idx:end_idx].to(device)
    batch_token_type_ids = tokenized_record['token_type_ids'][start_idx:end_idx].to(device)

    with torch.no_grad():
        embedding = model(input_ids=batch_input_ids,
                        attention_mask=batch_attention_mask,
                        token_type_ids=batch_token_type_ids)
        embedding = embedding.last_hidden_state.mean(dim=1).cpu()
    
    taxonomy_embeddings.append(embedding)

## Save embeddings

In [ ]:
records_embeddings = torch.cat(records_embeddings, dim=0)
print(records_embeddings.shape)
torch.save(records_embeddings, DATA_PATH + 'data/embeddings/bert_records_embeddings.pt')

torch.Size([6239, 1024])


In [ ]:
taxonomy_embeddings = torch.cat(taxonomy_embeddings, dim=0)
print(taxonomy_embeddings.shape)
torch.save(taxonomy_embeddings, DATA_PATH + 'data/embeddings/bert_taxonomy_embeddings.pt')

torch.Size([192, 1024])
